In [ ]:

# Sys imports
import os
import sys
os.environ["KERAS_BACKEND"] = "jax"

# Training API
import keras as keras
from keras import layers
import keras_nlp

# Data manipulation and exploration
import numpy as np
from scipy.stats import shapiro, kurtosis

# Data visualization
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

# NLP
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import jax.numpy as jnp

# MLOps API
import mlflow
mlflow.set_tracking_uri('http://localhost:5000')


# EDA

## Data loading
There are 72M titles and views in the data used.

In [ ]:
from src.scripts.dataloader import parse_jsonl_optimized

# Example usage: Import only the first 1000 lines from the file
file_path = '/mnt/datassd/train_data.jsonl'

# Pass None if you want ALL lines
num_lines_to_import = None 

titles, view_counts = parse_jsonl_optimized(file_path, num_lines_to_import)

## Create $Log_{10}(Data)$ and create tokenizer

When views are analyzed on their own, the MSE loss function does not make a lot of sense, there is a lot of variation in the data.

What's why we take $log_{10}$ to get the order of magnitude of the views ($10^x$).

In [ ]:

from transformers import BertTokenizer, BertModel
from concurrent.futures import ThreadPoolExecutor

# Load BERT tokanizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

Encoding all 70M datapoints takes ~45mins

In [ ]:
""" SINGLE THREADED  """
encoded_inputs = [tokenizer.encode(title, add_special_tokens=True) for title in tqdm(titles, total=len(titles), desc="Encoding")]
del titles

In the context of title engagement, UPPER CASED titles correlate with engagement, it is important to use a CASED tokenizer.

In [ ]:
# Testing cased inputs in tokenization
titulos_raros = [r'IS THIS TOKEN Cased?']
print(tokenizer.tokenize(titulos_raros[0]))
tokenizer.encode(titulos_raros[0], add_special_tokens=True)


In [ ]:
log_view_count = np.where(np.log10(view_counts) == -np.inf, 0, np.log10(view_counts))

# Bin the view counts in ranges 0-1, 1-2, 2-3, 3-4, 4-5, 5-6, 6-7, 7-8, 8+
def bin_data(views, a, b, n_bins):
    # Generate bin edges from a to b with n_bins
    if b > a and n_bins > 1:
        bin_edges = np.linspace(a, b, n_bins - 1)
    else:
        raise ValueError("Ensure that b > a and n_bins > 1")
    
    # Bin the data into categories
    view_bins = np.digitize(views, bin_edges, right=True)

    # Categorize data into n_bins categories
    view_categorical = to_categorical(view_bins, num_classes=n_bins)

    return view_categorical, view_bins


def reverse_bin_data(categorical_views, a, b, n_bins):
    # Check for valid input
    if b <= a or n_bins <= 1:
        raise ValueError("Ensure that b > a and n_bins > 1")

    # Calculate the bin width
    bin_width = (b - a) / n_bins
    
    # Find the bin indices from the categorical data
    bin_indices = np.argmax(categorical_views, axis=1)
    
    # Calculate the representative value for each bin. Using the bin center as the representative value.
    representative_values = a + bin_width * (bin_indices + 0.5)

    return representative_values

# IMPORTANT, IT SETS THE SCALE
a = 0
b = 8
n_bins = 16


view_categorical, view_bins = bin_data(log_view_count, a, b, n_bins)

In [ ]:
log_view_count[10], reverse_bin_data(view_categorical, a, b, n_bins)[10],view_categorical[10]

## Data Normalization tests

First, let's see the distribution of view counts

In [ ]:
# See Y view count distribution


sns.histplot(log_view_count, kde=True)
plt.title("Log of View Count Distribution")
plt.xlabel("Log of View Count")
plt.ylabel("Frequency")
plt.show()

In [ ]:
shapiro_test_stat, shapiro_p_value = shapiro(log_view_count)
kurtosis_value = kurtosis(log_view_count, fisher=True)

shapiro_test_stat, shapiro_p_value, kurtosis_value

In [ ]:
sample_titles_i = 500
print(view_counts[sample_titles_i:sample_titles_i+5])
print(log_view_count[sample_titles_i:sample_titles_i+5])
titles[sample_titles_i:sample_titles_i+5]


In [ ]:

# Histogram the bins
sns.histplot(view_bins, kde=False)
plt.title("View Count Bins")
plt.show()

Now let's see the ditribution of lenghts of titles.

We need to have a cutoff at a certain token lenght. So let's visualize when it would be appropriate to cut titles off.

In [ ]:
sns.histplot([len(encoded_input) for encoded_input in encoded_inputs], bins=50)
plt.title("Histogram of tokenized title lengths")
plt.xlabel("Length of tokenized title")
plt.ylabel("Number of titles")
plt.show()

Probably at 40 tokens, the cutoff would preserve most of the information. (Remember YouTube titles have a 100 char limit)

In [ ]:
max_length = 40  # Ensure your data is adjusted accordingly

padded_inputs = pad_sequences(encoded_inputs, maxlen=max_length, padding='post', truncating='post')


In [ ]:
# Just making sure, all inputs are the same length of 40 
sns.histplot([len(padded_input) for padded_input in padded_inputs], bins=50)
plt.title("Histogram of padded title lengths")
plt.xlabel("Length of padded title")
plt.ylabel("Number of titles")
plt.show()

Some samples to see the PADs

In [ ]:
sample = [tokenizer.decode(padded_input) for padded_input in padded_inputs[90:100]]
sample

In [ ]:
X_t = padded_inputs
del padded_inputs
Y_t = view_categorical
del view_categorical

vocab_size = 30522  # Adjusted to match BERT's vocabulary size for bert-base-cased
embedding_dim = 768  # Standard BERT-base embedding dimension
transformer_encoder_layers = 1  # Increase this for a larger model
num_heads = 12  # Increase this for a larger model
intermediate_dim = 768*2  # Dimensionality of the encoder layers and the pooler layer
dropout_rate = 0.1  # Dropout rate for regularization


In [ ]:
# Input layer
inputs = keras.Input(shape=(max_length,), dtype='int32')

# Embedding layer
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=vocab_size, 
    sequence_length=max_length, 
    embedding_dim=embedding_dim,
)

x = embedding_layer(inputs)

# Transformer blocks
for _ in range(transformer_encoder_layers):
    encoder = keras_nlp.layers.TransformerEncoder(
        num_heads=num_heads,
        intermediate_dim=intermediate_dim,
        activation='relu',
        dropout=dropout_rate,
    )
    x = encoder(x)

# Pooling and output layers
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(embedding_dim, activation='relu')(x)
outputs = layers.Dense(n_bins, activation='softmax')(x)  # 8 classes for the bins

# Model compilation
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model's architecture
model.summary()

In [ ]:
X_t.shape, Y_t.shape

In [ ]:


def batch_loader(X, Y, batch_size):
    num_samples = X.shape[0]
    indices = np.arange(num_samples)
    np.random.shuffle(indices)  # Shuffle if you want to randomize the batches
    
    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_indices = indices[start_idx:end_idx]
        yield X[batch_indices], Y[batch_indices]

class MlflowCallbackLogPerBatch(mlflow.keras_core.MLflowCallback):
    def __init__(self, run, log_every_n_steps):
        super().__init__(run=run)
        self.log_every_n_steps = log_every_n_steps
        self._log_step = 0

    def on_batch_end(self, batch, logs=None):
        if logs is None:
            return
        if (batch + 1) % self.log_every_n_steps == 0:
            # Extract scalar values from the numpy arrays
            loss = logs['loss'][0].item()  # Convert numpy array to Python scalar
            accuracy = logs['loss'][1].item()  # Convert numpy array to Python scalar
            
            # Log the metrics using MLflow
            mlflow.log_metrics({'loss': loss, 'accuracy': accuracy}, step=self._log_step)
            self._log_step += 1



# Start MLflow run and train the model
with mlflow.start_run() as run:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    batch_size = 32
    epochs = 5
    callback = MlflowCallbackLogPerBatch(run=run, log_every_n_steps=5)

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        # Setup tqdm for the batch loop
        with tqdm(total=len(X_t) // batch_size) as pbar:
            for batch, (X_batch, Y_batch) in enumerate(batch_loader(X_t, Y_t, batch_size)):
                history = model.train_on_batch(X_batch, Y_batch)
                logs = {'loss': history}
                callback.on_batch_end(batch=batch, logs=logs)
                # Update the progress bar
                pbar.update(1)
        # Optionally validate your model here if needed

    mlflow.end_run()


In [ ]:
# Save model
model.save('model.keras')

# Evaluation

In [ ]:
# Load the eval input
eval_file_path = '/mnt/datassd/train_data.jsonl'
titles_eval, view_counts_eval = parse_jsonl_optimized(eval_file_path, num_lines_to_import*0.005)
encoded_evals = [tokenizer.encode(title_ev, add_special_tokens=True) for title_ev in tqdm(titles_eval, total=len(titles_eval), desc="Encoding")]
padded_evals = pad_sequences(encoded_evals, maxlen=max_length, padding='post', truncating='post')

# eval output
log_view_count_eval = np.where(np.log10(view_counts_eval) == -np.inf, 0, np.log10(view_counts_eval))
view_evals_test, view_evals_bins = bin_data(log_view_count_eval, a, b, n_bins)

In [ ]:
Y_e = view_evals_test
X_e = padded_evals

In [ ]:
Y_pred = model.predict(X_e)
actual = log_view_count_eval
predicted = reverse_bin_data(Y_pred, a, b, n_bins)

In [ ]:

# actual = []
# predicted = []

# for i in range(len(Y_pred)):
#     pr = reverse_bin_data(Y_pred[i], a, b, n_bins)
#     ac = reverse_bin_data(Y_e[i], a, b, n_bins)
#     print(f"Predicted: {pr}, Real: {ac}")
#     actual.append(ac)
#     predicted.append(pr)

len(predicted)

In [ ]:
# Make scatter of predicted vs actual
plt.scatter(actual, predicted,alpha=0.005, s=10, cmap='viridis')
# Make a line
plt.plot([0, 10], [0, 10], color='red')
plt.title("Predicted vs Actual Score")
plt.xlabel("Actual Score View Count")
plt.ylabel("Predicted Score View Count")
plt.show()

In [ ]:
sns.histplot(predicted, kde=False)
plt.title("Predicted Score View Count")

In [ ]:
# Arange titles by highest predicted view count
sorted_titles = [title for _, title in sorted(zip(predicted, titles_eval), reverse=True)]
sorted_titles[:50], sorted_titles[-50:]

# corresponding view counts
sorted_view_counts = [view_count for _, view_count in sorted(zip(predicted, view_counts_eval), reverse=True)]


# Do both in one line
sorted_titles, sorted_view_counts = zip(*[(title, view_count) for _, title, view_count in sorted(zip(predicted, titles_eval, view_counts_eval), reverse=True, key=lambda x: x[0])])
sorted_titles[:5], sorted_titles[-5:], sorted_view_counts[:5], sorted_view_counts[-5:]
